In [ ]:
for rho=[1,500,1000,5000,10000]

    Q(1,1)=rho; [K,P]=lqr(A,B,Q,R);
    
    A_=[A B; C 0];
    xu=inv(A_)*[0 0 0 0 1]';
    
    rss=1;    
    xss=xu(1:4);uss=xu(5)
    Nx=xss/rss
    Nu=uss/rss
    Nbar=Nu+K*Nx
    
    Ace = [(A-B*K) (B*K);zeros(size(A)) (A-L*C)];
    Bce = [B*Nbar; zeros(size(B))];
    Cce = [Cc zeros(size(Cc))];
    Dce = [0;0];
    
    states = {'x' 'x_dot' 'phi' 'phi_dot' 'e1' 'e2' 'e3' 'e4'};
    inputs = {'r'};
    outputs = {'Angle(rad)'; 'Cart position(m)'};
    
    sys_cl = ss(Ace,Bce,Cce,Dce,'statename',states,'inputname',inputs,'outputname',outputs);
    
    figure(9)
    
    step(sys_cl); grid on; hold on
    title('Step response with different weights in Q(1,1) and Nbar correction')

end

In [ ]:
Q = obsv(A, [0, 0, 1, 0])
iQ = inv(Q)
q = iQ[:,3]
QQ = array([q, mul_mat(A,q), mul_mat(A,A,q), mul_mat(A,A,A,q)]).T
T = QQ.copy()
Ao = mul_mat(inv(T), A, T)
Bo = dot(inv(T), B)
Co = dot(C, T)

# Nbar
Cn = array([[0, 0, 1, 0]])
Dn = array([[0]])
a1_ = concatenate((A, B), axis=1)
a2_ = concatenate((Cn, Dn), axis=1)
AA = concatenate((a1_, a2_), axis=0)

# set point é 0.2
rss = 0.2
t = array([[0, 0, 0, 0, rss]]).T
ux = dot(inv2(AA),t)
xss = aux[0:4,:]
uss = aux[4,:]
Nx = xss/rss
Nu = uss/rss
Nbar = Nu + dot(K,Nx)
Nbar = float(Nbar)

# construindo At
a11 = A - dot(B,K)
a12 = dot(B,K)
a21 = zeros(A.shape)
a22 = A - dot(L2.T,Cn)
a1_ = concatenate((a11, a12), axis=1)
a2_ = concatenate((a21, a22), axis=1)
Ace = concatenate((a1_, a2_), axis=0)
# construindo Bt
b11 = B*Nbar
b21 = zeros(B.shape)
Bce = concatenate((b11, b21), axis=0)
# construindo Ct
c11 = C
c12 = zeros(C.shape)
Cce = concatenate((c11, c12), axis=1)

Dc2 = 0
sys_ss = ss(Ace,Bce,Cce,Dc2)



In [ ]:
f, axes = plt.subplots(2, 1)

sns.lineplot(x='T', y='yout0', hue='label', data=resp_df['ci'], ax=axes[0])
sns.lineplot(x='T', y='yout1', hue='label', data=resp_df['ci'], ax=axes[1])

axes[0].legend(loc="upper right")
axes[1].legend(loc="upper right")

axes[0].set_xlabel('')

axes[0].set_ylabel('θ (rad)'+'\n')
axes[1].set_ylabel('posição (m)'+'\n')

f.suptitle('Resposta ao impulso (CI nulas)'+'\n')

plt.xlabel('\nTempo (s)')
plt.show()

In [ ]:
f, axes = plt.subplots(2, 1)

sns.lineplot(x='T', y='yout0', hue='label', data=resp_df['wave'], ax=axes[0])
sns.lineplot(x='T', y='yout1', hue='label', data=resp_df['wave'], ax=axes[1])

axes[0].legend(loc="upper right")
axes[1].legend(loc="lower right")

axes[0].set_xlabel('')

axes[0].set_ylabel('θ (rad)'+'\n')
axes[1].set_ylabel('posição (m)'+'\n')

f.suptitle('Resposta a uma onda quadrada (CI nulas)'+'\n')

plt.xlabel('\nTempo (s)')
plt.show()

In [ ]:
class Systems():
    
    def __init__(self, parametros: list):
        """
        :param parametros: Dict no template [{'Q11':1, 'Q33':10, 'R':5}, {'Q11':100, 'Q33':1, 'R':1}, {'Q11':1, 'Q33':1, 'R':999}]
        """
        self.Qc = None
        self.R = None
        self.params = parametros
        self.sys = {}
    
    def reset_params(self):
        self.Qc = dot(C.T, C)
        self.R = 1
    
    def set_param(self, param: dict):
        self.Qc[0,0] = param['Q11']
        self.Qc[2,2] = param['Q33']    
        self.R = param['R']
        
        return f'Q11 {self.Qc[0,0]}, Q33 {self.Qc[2,2]}, R {self.R}'
    
    def get_new_sys(self):
        """Apenas LQR."""
        K, S, E  = lqr(A, B, self.Qc, self.R)

        Ac = A-dot(B,K)
        Bc = B
        Cc = C
        Dc = D
        
        sys_ss = ss(Ac,Bc,Cc,Dc2)
        
        return sys_ss

    def get_new_sys_obs(self):
        """LQR + Obs."""
        K, S, E  = lqr(A, B, self.Qc, self.R)

#         Q = obsv(A, [0, 0, 1, 0])
#         iQ = inv(Q)
#         q = iQ[:,3]
#         QQ = array([q, mul_mat(A,q), mul_mat(A,A,q), mul_mat(A,A,A,q)]).T
#         T = QQ.copy()
#         Ao = mul_mat(inv(T), A, T)
#         Bo = dot(inv(T), B)
#         Co = dot(C, T)

        # Nbar
        Cn = array([[0, 0, 1, 0]])
        Dn = array([[0]])
        a1_ = concatenate((A, B), axis=1)
        a2_ = concatenate((Cn, Dn), axis=1)
        AA = concatenate((a1_, a2_), axis=0)

        # set point é 0.2
        rss = 0.2
        t = array([[0, 0, 0, 0, rss]]).T
        ux = dot(inv2(AA),t)
        xss = aux[0:4,:]
        uss = aux[4,:]
        Nx = xss/rss
        Nu = uss/rss
        Nbar = Nu + dot(K,Nx)
        Nbar = float(Nbar)

        # construindo At
        a11 = A - dot(B,K)
        a12 = dot(B,K)
        a21 = zeros(A.shape)
        a22 = A - dot(L2.T,Cn)
        a1_ = concatenate((a11, a12), axis=1)
        a2_ = concatenate((a21, a22), axis=1)
        Ace = concatenate((a1_, a2_), axis=0)
        # construindo Bt
        b11 = B*Nbar
        b21 = zeros(B.shape)
        Bce = concatenate((b11, b21), axis=0)
        # construindo Ct
        c11 = C
        c12 = zeros(C.shape)
        Cce = concatenate((c11, c12), axis=1)
        # Dt
        Dc2 = 0
        
        sys_ss = ss(Ace,Bce,Cce,Dc2)

        return sys_ss
    
    def run(self, obs=True):
        for param in self.params:
            self.reset_params()
            label = self.set_param(param)
            
            if obs:
                sys = self.get_new_sys_obs()
            else:
                sys = self.get_new_sys()
                
            self.sys.update({label:sys})
        
        return self.sys
    

class Responses():
    
    def __init__(self, sys: dict):
        """
        :para, sys: Dict que contém os sistemas que serão testas sob as respostas.
        """
        self.sys = sys
        self.step = {}
        self.ci = {}
        self.wave = {}
        self.t = arange(0, 30, .01)
        self.Dc2 = 0
        self.xo = [0.1, 0, 1, 0, 0, 0, 0, 0]
        self.t_square = arange(0, 180, 0.1)
        self.u_square = square(2 * pi * 1/90 * self.t_square)    
    
    def run(self):
        for label, sys in self.sys.items():
            T, yout, xout = forced_response(sys, self.t, U=0.2)
            self.step.update({label:{'T':T, 'yout0':yout[0], 'yout1':yout[1]}})
            
            T, yout = impulse_response(sys, T=self.t, X0=self.xo)
            self.ci.update({label:{'T':T, 'yout0':yout[0], 'yout1':yout[1]}})
            
            T, yout, xout = forced_response(sys, self.t_square, U=self.u_square)
            self.wave.update({label:{'T':T, 'yout0':yout[0], 'yout1':yout[1]}})
        
        return {'step': self.step, 'ci': self.ci, 'wave': self.wave}
        
        
class TidyData():
    
    def __init__(self, resp: dict):
        """
        :param resp: Dict que contém as respostas dos sistemas.
        """    
        self.resp = resp
        self.final = {'step': None, 'ci': None, 'wave': None}
    
    def tidy(self, name_resp: str):
        """:param name_resp: Str que seja ['step', 'ci' ou 'wave']."""
        resp_target = self.resp[name_resp]

        primeiro = True

        for label, values in resp_target.items():

            df_aux = pd.DataFrame(values)
            df_aux['label'] = label

            if primeiro:
                df = df_aux.copy()
                primeiro = False
            else:
                df = pd.concat([df, df_aux])

            #print(df.shape)
        
        self.final[name_resp] = df
    
    def run(self) -> dict:
        for resposta in ['step', 'ci', 'wave']:
            self.tidy(resposta)
        
        return self.final        

    
class Visualizacao():
    
    def __init__(self, resp_df: dict):
        """:param resp_df: Dict com as respostas em pd.DataFrame"""
        sns.set_theme(style="darkgrid")
        
        self.resp_df = resp_df
        self.resp = None
        self.pos_leg0 = "upper right"
        self.pos_leg1 = "lower right"
        self.titulo = 'Resposta ao degrau (CI nulas)'
    
    def set_param(self, type_resp: str):
        """:param type_resp: Str é o tipo de resposta, exe: ['step', 'ci', 'wave']"""
        self.resp = type_resp
        
        if type_resp == 'step':
            self.pos_leg0 = "upper right"
            self.pos_leg1 = "lower right"
            self.titulo = 'Resposta ao degrau (CI nulas)'      
        elif type_resp == 'ci':
            self.pos_leg0 = "upper right"
            self.pos_leg1 = "upper right"
            self.titulo = 'Resposta ao impulso (CI não nulas)'  
        else:
            self.pos_leg0 = "upper right"
            self.pos_leg1 = "lower right"
            self.titulo = 'Resposta a uma onda quadrada (CI nulas)'              
    
    
    def show(self):
        f, axes = plt.subplots(2, 1)

        sns.lineplot(x='T', y='yout0', hue='label', data=self.resp_df[self.resp], ax=axes[0])
        sns.lineplot(x='T', y='yout1', hue='label', data=self.resp_df[self.resp], ax=axes[1])

        axes[0].legend(loc=self.pos_leg0)
        axes[1].legend(loc=self.pos_leg1)

        axes[0].set_xlabel('')

        axes[0].set_ylabel('θ (rad)'+'\n')
        axes[1].set_ylabel('posição (m)'+'\n')

        f.suptitle(self.titulo+'\n')

        plt.xlabel('\nTempo (s)')
        plt.show()
        
    